In [1]:
import rioxarray
import rasterio as rio
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
from pathlib import Path

In [2]:
vi = 'ndvi'
run = 'f6'
outf = f'/home/geodata/Clientes/0FARMS/FabioRicardi-Barreiras_BA/raster/{run}'
folder = '/home/geodata/Clientes/0FARMS/FabioRicardi-Barreiras_BA/CARS-Fazenda_Savana.gpkgnc/f4/'

outf = f'/home/geodata/Clientes/0FARMS/MG-3102605-B4D344DBFD874F44906FCC0A5E0DCE36/raster/{run}/'
folder = '/home/geodata/Clientes/0FARMS/MG-3102605-B4D344DBFD874F44906FCC0A5E0DCE36/nc/f5/'


Path(outf).mkdir( parents = True, exist_ok = True)

ff = sorted(glob(folder+f'{vi}*.nc'))
print(ff)

['/home/geodata/Clientes/0FARMS/MG-3102605-B4D344DBFD874F44906FCC0A5E0DCE36/nc/f5/ndvi_1985-07-02-1986-05-02_9.nc', '/home/geodata/Clientes/0FARMS/MG-3102605-B4D344DBFD874F44906FCC0A5E0DCE36/nc/f5/ndvi_1986-07-21-1987-06-06_16.nc', '/home/geodata/Clientes/0FARMS/MG-3102605-B4D344DBFD874F44906FCC0A5E0DCE36/nc/f5/ndvi_1987-06-22-1988-06-08_18.nc', '/home/geodata/Clientes/0FARMS/MG-3102605-B4D344DBFD874F44906FCC0A5E0DCE36/nc/f5/ndvi_1988-06-24-1989-05-26_13.nc', '/home/geodata/Clientes/0FARMS/MG-3102605-B4D344DBFD874F44906FCC0A5E0DCE36/nc/f5/ndvi_1989-06-27-1990-06-14_11.nc', '/home/geodata/Clientes/0FARMS/MG-3102605-B4D344DBFD874F44906FCC0A5E0DCE36/nc/f5/ndvi_1990-06-30-1991-06-01_13.nc', '/home/geodata/Clientes/0FARMS/MG-3102605-B4D344DBFD874F44906FCC0A5E0DCE36/nc/f5/ndvi_1991-08-04-1992-06-19_8.nc', '/home/geodata/Clientes/0FARMS/MG-3102605-B4D344DBFD874F44906FCC0A5E0DCE36/nc/f5/ndvi_1992-07-05-1993-06-06_15.nc', '/home/geodata/Clientes/0FARMS/MG-3102605-B4D344DBFD874F44906FCC0A5E0DCE3

In [3]:
df = xr.open_mfdataset(ff, combine = 'nested', chunks = 2048, concat_dim = 'time', coords = 'minimal');

if len(df.time) != len(np.unique(df.time.values)):
    seen = set()
    dupes = [x for x in df.time.values if x in seen or seen.add(x)] 
    print(dupes)
else:
    print('your time-series has no dupes, good job!')

your time-series has no dupes, good job!


In [ ]:
plt.hist(np.ravel(df['NDVI'].values), bins = 100);

In [4]:
yr = (2018,2024)
mr = (3,6)
season = 'gs36'
qs = [ .10 , .25 , .50 , .75 , .90 , .99 ]

In [5]:
for ano in range( yr[0] , yr[1]):
    t0 = f'{ano}-{str(mr[0]).zfill(2)}'
    t1 = f'{ano}-{str(mr[1]).zfill(2)}'


    df2 = df.sel( time = slice( t0 , t1 ))
    if len(df2.time.values)>1:
        for q in qs:
            print(f'... quantile {q}')
            quantile = df2.quantile(q, dim = 'time') * 1000
            quantile = quantile.astype('int16')
            quantile.rio.to_raster(f'{outf}{vi}_{season}_{t0}_{t1}_{q}.tif')
            print(f'<< saved: {outf}{vi}_{season}_{t0}_{t1}_{q}.tif')
    else:
        print(f'nothing for {ano}')

... quantile 0.1


ValueError: dimension time on 0th function argument to apply_ufunc with dask='parallelized' consists of multiple chunks, but is also a core dimension. To fix, either rechunk into a single array chunk along this dimension, i.e., ``.chunk(dict(time=-1))``, or pass ``allow_rechunk=True`` in ``dask_gufunc_kwargs`` but beware that this may significantly increase memory usage.

### Quantiles
#### Select month and year range

In [ ]:
# payload
yr = (2013,2024)
mr = (3,6)
season = 'gs36'
qs = [ .10 , .25 , .50 , .75 , .90 , .99 ]

In [6]:
# para inicio a meio do ano
for ano in range( yr[0] , yr[1]):
    t0 = f'{ano}-{str(mr[0]).zfill(2)}'
    t1 = f'{ano}-{str(mr[1]).zfill(2)}'
    print(t0,t1)

    df2 = df.sel( time = slice( t0 , t1 ))
    df2 = df2.chunk(dict(time=-1))
    if len(df2.time.values)>1:
        for q in qs:
            print(f'... quantile {q}')
            quantile = df2.quantile(q, dim = 'time') * 1000
            quantile = quantile.astype('int16')
            quantile.rio.to_raster(f'{outf}{vi}_{season}_{t0}_{t1}_{q}.tif')
            print(f'<< saved: {outf}{vi}_{season}_{t0}_{t1}_{q}.tif')
    else:
        print(f'nothing for {ano}')

2018-03 2018-06
... quantile 0.1


/home/mariano/geo/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/home/mariano/geo/lib/python3.10/site-packages/dask/array/chunk.py:278: RuntimeWarning: invalid value encountered in cast
  return x.astype(astype_dtype, **kwargs)


<< saved: /home/geodata/Clientes/0FARMS/MG-3102605-B4D344DBFD874F44906FCC0A5E0DCE36/raster/f6/ndvi_gs36_2018-03_2018-06_0.1.tif
... quantile 0.25


/home/mariano/geo/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/home/mariano/geo/lib/python3.10/site-packages/dask/array/chunk.py:278: RuntimeWarning: invalid value encountered in cast
  return x.astype(astype_dtype, **kwargs)


<< saved: /home/geodata/Clientes/0FARMS/MG-3102605-B4D344DBFD874F44906FCC0A5E0DCE36/raster/f6/ndvi_gs36_2018-03_2018-06_0.25.tif
... quantile 0.5


...

<< saved: /home/geodata/Clientes/0FARMS/MG-3102605-B4D344DBFD874F44906FCC0A5E0DCE36/raster/f6/ndvi_gs36_2020-03_2020-06_0.1.tif
... quantile 0.25


/home/mariano/geo/lib/python3.10/site-packages/dask/array/chunk.py:278: RuntimeWarning: invalid value encountered in cast
  return x.astype(astype_dtype, **kwargs)
/home/mariano/geo/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


<< saved: /home/geodata/Clientes/0FARMS/MG-3102605-B4D344DBFD874F44906FCC0A5E0DCE36/raster/f6/ndvi_gs36_2020-03_2020-06_0.25.tif
... quantile 0.5


/home/mariano/geo/lib/python3.10/site-packages/dask/array/chunk.py:278: RuntimeWarning: invalid value encountered in cast
  return x.astype(astype_dtype, **kwargs)
/home/mariano/geo/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


<< saved: /home/geodata/Clientes/0FARMS/MG-3102605-B4D344DBFD874F44906FCC0A5E0DCE36/raster/f6/ndvi_gs36_2020-03_2020-06_0.5.tif
... quantile 0.75
